In [34]:
from qdrant_client import QdrantClient

qdrant_client = QdrantClient(url="http://localhost:6333")

from langchain_gigachat import GigaChatEmbeddings
import os

giga_embeddings =  GigaChatEmbeddings(credentials=os.environ["GIGA_CREDENTIALS"], model="EmbeddingsGigaR", verify_ssl_certs=False)

In [2]:
from  langchain_qdrant import Qdrant


In [36]:
from langchain_core.documents import Document
import pandas as pd

pdf_df = pd.read_csv("data/pdf_df.csv", index_col=0)
threads_df = pd.read_csv("data/threads.csv", index_col=0)
vk_chunks_df = pd.read_csv("data/vk_chunks_df.csv",  index_col=0)

In [42]:
vector_store = Qdrant(client=qdrant_client,  embeddings=giga_embeddings)
vector_store = vector_store.from_documents([Document(metadata=a.to_dict(), page_content=a.chunk_text) for i, a in pdf_df.iterrows()], embedding=giga_embeddings)

In [51]:
vector_store.

[Document(metadata={'doc_name': 'Правила Приема(Министерские)', 'chunk_id': 2, 'chunk_text': 'федеральных государственных органов, указанных в части 1 статьи 81 Федерального закона от 29 декабря 2012 г. № 273-ФЗ «Об образовании в Российской Федерации» 2 (далее - Федеральный закон № 273-ФЗ), устанавливаются указанными федеральными государственными органами1. 4. Прием на обучение осуществляется на первый курс. 5. Прием на обучение осуществляется: по программам бакалавриата и программам специалитета - при наличии у поступающих среднего общего образования, или среднего профессионального образования, или высшего образования; по программам магистратуры - при наличии у поступающих высшего образования. Наличие образования подтверждается документами об образовании или об образовании и о квалификации, выдаваемыми лицам, успешно прошедшим государственную итоговую аттестацию либо итоговую аттестацию (далее - документы об образовании). 6. Поступающим необходимо иметь: 1) для поступления на обучение

In [49]:
threads_df

,thread_id,root_global_id,turns,dialog_plain,dialog_annotated,participants,global_ids
0,1,chat2021_6,1,Порядок распределения по кафедрам/направлениям...,[Институт №8 МАИ 2021 (6)] Порядок распределен...,"[('Институт №8 МАИ 2021', 6)]",['chat2021_6']
1,2,chat2021_7,1,Порядок получения общежития При зачислении мес...,[Институт №8 МАИ 2021 (7)] Порядок получения о...,"[('Институт №8 МАИ 2021', 7)]",['chat2021_7']
2,3,chat2021_10,1,Возможность заселения в одну комнату знакомых/...,[Институт №8 МАИ 2021 (10)] Возможность заселе...,"[('Институт №8 МАИ 2021', 10)]",['chat2021_10']
3,4,chat2021_11,1,Перевод между направлениями 8 института. Перев...,[Институт №8 МАИ 2021 (11)] Перевод между напр...,"[('Институт №8 МАИ 2021', 11)]",['chat2021_11']
4,5,chat2021_12,1,Программа обучения программированию на 1 курсе...,[Институт №8 МАИ 2021 (12)] Программа обучения...,"[('Институт №8 МАИ 2021', 12)]",['chat2021_12']
...,...,...,...,...,...,...,...
42977,42978,chat2024_70489,1,"👀 Студенты, а вы помните крышесносную приемную...","[Максим Приемка (70489)] 👀 Студенты, а вы помн...","[('Максим Приемка', 70489)]",['chat2024_70489']
42978,42979,chat2024_70490,1,Еще 5-ть мест на экскурсию для перваков 8 инст...,[Максим Приемка (70490)] Еще 5-ть мест на экск...,"[('Максим Приемка', 70490)]",['chat2024_70490']
42979,42980,chat2024_70505,1,👀 Мы знаем как волнительна и тревожна для перв...,[Максим Приемка (70505)] 👀 Мы знаем как волнит...,"[('Максим Приемка', 70505)]",['chat2024_70505']
42980,42981,chat2024_70507,2,👀 Мы знаем как волнительна и тревожна для перв...,[(Архив 2024) 8 институт МАИ - это IT 2024 (70...,"[('(Архив 2024) 8 институт МАИ - это IT 2024',...","['chat2024_70507', 'chat2024_70509']"


In [20]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.tools import tool

import os


llm = ChatOpenAI(
    model="gpt://" + os.environ["folder_id"] + "/yandexgpt/rc",
    api_key=os.environ["api_key"],
    base_url="https://llm.api.cloud.yandex.net/v1"
)

@tool
def search(query:str) -> str:
    """Ищет информацию нужную пользователю"""
    return f"Крутая  информация про {query}"

In [22]:
llm_wth_tools = llm.bind_tools(tools=[search])

In [25]:
search.invoke(llm_wth_tools.invoke("Найди инфу про бобров").tool_calls[0])

ToolMessage(content='Крутая  информация про бобры', name='search', tool_call_id='search')